In [1]:
# Importing necessary modules
import re
import spacy
from spacy import displacy
#python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [2]:
# Setting up notebook to display multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Setting list of directions, as would be obtained in web_scraping.ipynb
directions = ['Preheat the oven to 450 degrees F',
              'Pulse the carrots, celery and shallots in a food processor until coarsely chopped',
              'Heat 1 tablespoon of the olive oil in a large nonstick skillet over medium high',
              'Add the chopped vegetables and cook, stirring frequently, until light golden and soft, 8 to 10 minutes',
              '(Add a splash of water if the mixture begins to stick',
              ') Add the ground beef and 4 teaspoons of the tomato paste and cook, breaking the mixture up with a wooden spoon, until browned, about 4 minutes',
              'Add 1/2 cup water, the parsley, raisins, 1 tablespoon of the vinegar and 3/4 teaspoon salt',
              'Bring to a simmer and cook until most of the water is absorbed and the mixture gets saucy, about 1 minute',
              'Let cool slightly',
              'Meanwhile, toss the pepper halves with the remaining 1/2 tablespoon oil in a microwave-safe bowl',
              'Cover with plastic wrap and microwave until the peppers are pliable, 10 to 12 minutes',
              'Carefully uncover the bowl and pour out any liquid that has accumulated',
              'Stir the couscous into the beef mixture',
              'Whisk together the remaining 2 tablespoons tomato paste, 1 tablespoon vinegar and 3/4 cup water in the bottom of a large baking dish',
              'Carefully transfer the peppers to the baking dish cut-side up and fill each pepper with the couscous mixture',
              'Sprinkle with the cheese',
              'Cover with foil and bake until the peppers are tender and the stuffing is hot, 20 to 25 minutes',
              'Serve warm or at room temperature drizzled with the tomato cooking liquid']

In [4]:
# Creating a Doc object from the directions
direction_doc = nlp(directions[0])

# Dependency parsing the directions
for token in direction_doc:
    print(token.text)
    print(token.tag_)
    print(token.head.text)
    print(token.dep_)
    print("\n")
displacy.render(direction_doc, style = "dep")

Preheat
VB
Preheat
ROOT


the
DT
oven
det


oven
NNP
Preheat
dobj


to
IN
Preheat
prep


450
CD
degrees
nummod


degrees
NNS
to
pobj


F
NN
Preheat
dobj


